In [65]:

import numpy as np 
import random
import time
from bokeh.plotting import figure, show, output_notebook 
from bokeh.models import ColumnDataSource

# Configure Bokeh to show plots in the notebook
output_notebook()

# Berlin52 coordinates
coordinates = np.array([
[565, 575], [25, 185], [345, 750], [945, 685], [845, 655], [880, 660],
[25, 230], [525, 1000], [580, 1175], [650, 1130], [1605, 620], [1220, 580], 
[1465, 200], [1530, 5], [845, 680], [725, 370], [145, 665], [415, 635], 
[510, 875], [560, 365], [300, 465], [520, 585], [480, 415], [835, 625], 
[975, 580], [1215, 245], [1320, 315], [1250, 400], [660, 180], [410, 250], 
[420, 555], [575, 665], [1150, 1160], [700, 580], [685, 595], [685, 610],
[770, 610], [795, 645], [720, 635], [760, 650], [475, 960], [95, 260], 
[875, 920], [700, 500], [555, 815], [830, 485], [1170, 65], [830, 610], 
[605, 625], [595, 360], [1340, 725], [1740, 245]
])

# Functions
def euclidean_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

def tour_cost(tour, points):
    cost = 0
    for i in range(len(tour)):
        cost += euclidean_distance(points[tour[i]], points [tour[(i + 1) % len(tour)]])
        return cost

def stochastic_two_opt(tour):
    a, b = random.sample(range(len(tour)), 2)
    if a > b:
        a, b = b, a
    new_tour = tour[:a] + tour[a:b][::-1] + tour[b:]
    return new_tour


Loading BokehJS ...

In [66]:

def construct_greedy_solution (points): 
    unvisited = list(range(len(points)))
    tour = [unvisited.pop(random.randint(0, len(unvisited) - 1))] 
    while unvisited:
        next_point = min(unvisited, key = lambda x:
        euclidean_distance(points[tour[-1]], points[x]))
        tour.append(next_point) 
        unvisited.remove(next_point)
    return tour

def local_search(tour, points):
    best_tour = tour
    best_cost = tour_cost(tour, points)
    for _ in range(100): # Fixed number of local search attempts
        new_tour = stochastic_two_opt(best_tour) 
        new_cost = tour_cost(new_tour, points)
        if new_cost < best_cost:
            best_tour, best_cost = new_tour, new_cost
    return best_tour, best_cost

In [67]:
def grasp(points, max_iterations, patience): 
    best_tour = None
    best_cost = float('inf')
    no_improvement_counter = 0  # Counter for iterations without improvement

    for i in range(max_iterations):
        initial_solution = construct_greedy_solution(points)
        local_optimum, local_cost = local_search(initial_solution, points)

        if local_cost < best_cost:
            best_tour = local_optimum
            best_cost = local_cost
            no_improvement_counter = 0  # Reset counter if improvement is found
            print(f"New best cost found: {best_cost} at iteration {i}")
        else:
            no_improvement_counter += 1

        # If no improvement is observed for 'patience' iterations, stop
        if no_improvement_counter >= patience:
            print(f"No improvement for {patience} iterations. Stopping early at iteration {i}.")
            break

    return best_tour, best_cost
    
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

def plot_tour_bokeh(tour, points):
    # Extracting coordinates from the tour
    x_coords = [points[i][0] for i in tour] + [points[tour[0]][0]]
    y_coords = [points[i][1] for i in tour] + [points[tour[0]][1]]
    
    # Create a ColumnDataSource
    source = ColumnDataSource(data=dict(x=x_coords, y=y_coords))
    
    # Create a Bokeh plot
    p = figure(title="Best Tour Found", 
               x_axis_label="X Coordinate", 
               y_axis_label="Y Coordinate", 
               width=800, height=600)
        
    p.title.align = "center"
    p.title.text_font_size = "20pt"
    
    # Add a line and circle glyphs to the plot
    p.line("x", "y", source=source, line_width=5, line_color="green") 
    p.scatter("x", "y", source=source, size=20, color="blue", alpha=0.6)
    
    # Show the plot
    show(p, notebook_handle=True)



In [68]:
# Running the GRASP algorithm
max_iterations = 1000
patience = 50  # Stop if no improvement in the best cost for 50 iterations
start_time = time.time()
best_tour, best_cost = grasp(coordinates, max_iterations, patience)
end_time = time.time()

plot_tour_bokeh(best_tour, coordinates)
print(f"Best cost found: {best_cost}")
print(f"Execution time: {end_time - start_time} seconds")


New best cost found: 21.213203435596427 at iteration 0
New best cost found: 15.811388300841896 at iteration 1
New best cost found: 15.0 at iteration 17
No improvement for 50 iterations. Stopping early at iteration 67.


Best cost found: 15.0
Execution time: 0.45633983612060547 seconds
